In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pickle
import os
import admix_prs

# Load data

In [173]:
PHENO_PATH = "/u/project/pasaniuc/kangchen/tmp/prs-1219/REAL-PHENO/all-pheno.csv"
df_pheno = pd.read_csv(PHENO_PATH, index_col=0)
df_pheno["PC1_Q"] = admix_prs.make_levels(df_pheno, stratify_col="PC1", n_level=5)
df_pheno["PC2_Q"] = admix_prs.make_levels(df_pheno, stratify_col="PC2", n_level=5)
df_pheno["AGE_Q"] = admix_prs.make_levels(df_pheno, stratify_col="AGE", n_level=5)

In [174]:
config = "hsq-0.25-pcausal-0.01-hermodel-uniform"

In [175]:
if os.path.exists("df_res_list.pkl"):
    with open("df_res_list.pkl", "rb") as f:
        df_res_list = pickle.load(f)
else:
    df_res_list = admix_prs.load_sim_data(config)
    with open("df_res_list.pkl", "wb") as f:
        pickle.dump(df_res_list, f)

In [180]:
for i_df in range(len(df_res_list)):
    df_prs = df_res_list[i_df].copy()
    df_prs["PHENO_G"] -= df_prs["PHENO"].mean()
    df_prs["PHENO"] -= df_prs["PHENO"].mean()
    df_prs["ID"] = df_prs.index
    df_prs["ID"] = df_prs["ID"].apply(lambda x: x.split("_")[0])
    df_prs.index = df_prs["ID"]
    df_prs = df_prs.drop(columns=["ID"])
    df_prs = df_prs.dropna()
    df_prs.index = df_prs.index.astype(int)
    df_prs["PHENO_STD"] = np.sqrt(df_info["PRS_STD"] ** 2 + 0.75)
    df_prs["PRS_LOW"] = df_prs["PRS_MEAN"] - 1.645 * df_prs["PRS_STD"]
    df_prs["PRS_UPP"] = df_prs["PRS_MEAN"] + 1.645 * df_prs["PRS_STD"]
    df_prs["PHENO_LOW"] = df_prs["PRS_MEAN"] - 1.645 * df_prs["PHENO_STD"]
    df_prs["PHENO_UPP"] = df_prs["PRS_MEAN"] + 1.645 * df_prs["PHENO_STD"]
    df_info = pd.merge(df_pheno, df_prs, left_index=True, right_index=True)
    
    res1 = admix_prs.stratify_calculate_r2(df_info, x_col="PHENO_G", y_col="PRS_MEAN", grp_col="PC1_Q")
    res2 = admix_prs.stratify_calculate_r2(df_info, x_col="PHENO", y_col="PRS_MEAN", grp_col="PC1_Q")
    res3 = admix_prs.stratify_calculate_r2(df_info, x_col="PHENO_G", y_col="PRS_MEAN", grp_col="PC2_Q")
    res4 = admix_prs.stratify_calculate_r2(df_info, x_col="PHENO", y_col="PRS_MEAN", grp_col="PC2_Q")
    res5 = admix_prs.stratify_calculate_r2(df_info, x_col="PHENO_G", y_col="PRS_MEAN", grp_col="SEX")
    res6 = admix_prs.stratify_calculate_r2(df_info, x_col="PHENO", y_col="PRS_MEAN", grp_col="SEX")
    res7 = admix_prs.stratify_calculate_r2(df_info, x_col="PHENO_G", y_col="PRS_MEAN", grp_col="AGE_Q")
    res8 = admix_prs.stratify_calculate_r2(df_info, x_col="PHENO", y_col="PRS_MEAN", grp_col="AGE_Q")
    res9 = admix_prs.eval_calibration(df_info, x_col="PHENO_G", lower_col="PRS_LOW", upper_col="PRS_UPP", grp_col="AGE_Q")
    res10 = admix_prs.eval_calibration(df_info, x_col="PHENO_G", lower_col="PRS_LOW", upper_col="PRS_UPP", grp_col="SEX")
    res11 = admix_prs.eval_calibration(df_info, x_col="PHENO_G", lower_col="PRS_LOW", upper_col="PRS_UPP", grp_col="PC1_Q")
    res12 = admix_prs.eval_calibration(df_info, x_col="PHENO_G", lower_col="PRS_LOW", upper_col="PRS_UPP", grp_col="PC2_Q")
    res13 = admix_prs.eval_calibration(df_info, x_col="PHENO", lower_col="PHENO_LOW", upper_col="PHENO_UPP", grp_col="AGE_Q")
    res14 = admix_prs.eval_calibration(df_info, x_col="PHENO", lower_col="PHENO_LOW", upper_col="PHENO_UPP", grp_col="SEX")
    res15 = admix_prs.eval_calibration(df_info, x_col="PHENO", lower_col="PHENO_LOW", upper_col="PHENO_UPP", grp_col="PC1_Q")
    res16 = admix_prs.eval_calibration(df_info, x_col="PHENO", lower_col="PHENO_LOW", upper_col="PHENO_UPP", grp_col="PC2_Q")
    res_li = [res1, res2, res3, res4, res5, res6, res7, res8, res9, res10, res11, res12, res13, res14, res15, res16]
    
    print("dataframe", i_df, "results: ")
    for res in res_li: print(res)

dataframe 0 results: 
                PC1_Q        R2    R2_std
0  (-19.271, -13.615]  0.730306  0.008726
1  (-13.615, -12.613]  0.730492  0.003772
2  (-12.613, -11.683]  0.738255  0.009167
3  (-11.683, -10.323]  0.732073  0.007706
4  (-10.323, 419.396]  0.655425  0.004165
                PC1_Q        R2    R2_std
0  (-19.271, -13.615]  0.174655  0.011344
1  (-13.615, -12.613]  0.185796  0.011888
2  (-12.613, -11.683]  0.180649  0.011908
3  (-11.683, -10.323]  0.175100  0.008686
4  (-10.323, 419.396]  0.188397  0.003833
               PC2_Q        R2    R2_std
0  (-282.318, 2.103]  0.758851  0.001919
1     (2.103, 3.217]  0.729418  0.006560
2      (3.217, 4.07]  0.724143  0.004555
3      (4.07, 5.039]  0.734761  0.005767
4    (5.039, 86.112]  0.574173  0.004386
               PC2_Q        R2    R2_std
0  (-282.318, 2.103]  0.222218  0.003481
1     (2.103, 3.217]  0.190980  0.008526
2      (3.217, 4.07]  0.186903  0.010168
3      (4.07, 5.039]  0.169975  0.007610
4    (5.039, 86.112]  0

1. stratify_calculate_r2 by PC1, PC2, AGE, SEX
- R2 different by PC1 and PC2, but not by AGE, SEX
- do this for both PHENO_G and PHENO

2. eval_calibration using PHENO_G and `[MEAN - 1.645 * STD, MEAN + 1.645 * STD]` 
- expect 90% coverage
3. eval_calibration using PHENO and `[MEAN - 1.645 * PHENO-STD, MEAN + 1.645 * PHENO-STD]`

- PHENO-G ~ PRS-MEAN + STD(PRS)
- PHENO ~ PRS-MEAN + STD(PRS + ENV)
simulated heritability is 0.25
var[y] = 1
hsq = var[gv] / (var[gv] + var[e]) = 0.25
var[e] = 0.75

STD(PRS + ENV) = sqrt(PRS-STD ** 2 + ENV-STD ** 2)

df["PHENO-STD"] = np.sqrt(df["PRS-STD"] ** 2 + 0.75)

ENV-STD ** 2 = 0.75 = 1 - 0.25